In [3]:
import os
import cv2
import numpy as np
import random
random.seed(66)

In [24]:
image_base = '/home/gzr/Data/hole_data/hole_dataset/hole/images/test/'
image_base_1 = '/home/gzr/Data/hole_data/hole_dataset/hole/images/train/'
image_base_2 = '/home/gzr/Data/hole_data/hole_dataset/hole/images/val/'
# point_base = image_base.replace('images','points')
data_type = 'data_'
w,h = 640, 480

def get_range_random(min_=0.9, max_=1.1):
    return random.random()*(max_-min_) + min_

def mkdir(path):
     if not os.path.exists(path):
        os.makedirs(path)





def get_external_contours_points(image):
    """
    :param image: (H,W)
    :return: (N,2)
    """
    assert (len(image.shape) == 2)
    ret, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    xcnts = np.vstack([x.reshape(-1, 2) for x in contours])

    # gap = xcnts.shape[0] // num_point
    # xcnts = xcnts[:num_point * gap:gap, :]
    return xcnts

def save_data(image_save_dir,image, image_mask, is_ellipse,data_type, cnt):
    point_save_dir = image_save_dir.replace('images','points')
    trans_save_dir = image_save_dir.replace('images','trans')
    mkdir(image_save_dir)
    mkdir(point_save_dir)
    mkdir(trans_save_dir)



    # 2. resize data
    scale = [w/image.shape[1],h/image.shape[0]] # x,y
    image_mask = cv2.resize(image_mask,dsize=(w,h))
    image = cv2.resize(image,dsize=(w,h))

    points = get_external_contours_points(image_mask)


    if is_ellipse:
        ellipse = cv2.fitEllipse(points)

        # 3.translate + scaling
        random_scale = get_range_random()
        center_ellipse = ellipse[0]
        axis_ellipse = (ellipse[1][0]*random_scale,ellipse[1][1]*random_scale)

        M = cv2.getRotationMatrix2D(center_ellipse,angle=0,scale=random_scale)
        raw_add = np.array([0,0,1])
        M = np.r_[M,[raw_add]]
        translate = [w/2-center_ellipse[0],h/2-center_ellipse[1]]
        M[0,2] += translate[0]
        M[1,2] += translate[1]
        M_trans = np.linalg.inv(M) #

        template = np.zeros_like(image_mask)
        ellipse = ((w/2,h/2),axis_ellipse,ellipse[2])

        cv2.ellipse(template,ellipse,(255,255,255),-1)
        cv2.imwrite(os.path.join(image_save_dir,str(cnt)+'.png'),image)
        cv2.imwrite(os.path.join(image_save_dir,str(cnt)+'_template.png'),template)
        np.save(os.path.join(trans_save_dir,str(cnt)+'_trans.npy'), M_trans)
        np.save(os.path.join(point_save_dir,str(cnt)+'_template.npy'),np.array([w/2,h/2]))
    else:
        center =  np.mean(points, axis=0)
        random_scale = get_range_random()
        M = cv2.getRotationMatrix2D(center,angle=0,scale=random_scale)
        raw_add = np.array([0,0,1])
        M = np.r_[M,[raw_add]]
        translate = [w/2-center[0],h/2-center[1]]
        M[0,2] += translate[0]
        M[1,2] += translate[1]
        M_trans = np.linalg.inv(M) #

        template = cv2.warpAffine(src=image_mask,M=M[0:2],dsize=(w,h))
        cv2.imwrite(os.path.join(image_save_dir,str(cnt)+'.png'),image)
        cv2.imwrite(os.path.join(image_save_dir,str(cnt)+'_template.png'),template)
        np.save(os.path.join(trans_save_dir,str(cnt)+'_trans.npy'), M_trans)
        np.save(os.path.join(point_save_dir,str(cnt)+'_template.npy'),np.array([w/2,h/2]))


def deal_sample(path_mask, path_image, is_ellipse,data_type, cnt):
    # 0. save path
    image_name = path_image.split('/')[-1]
    mask_name = path_mask.split('/')[-1]
    # point_name = path_point.split('/')[-1]
    data_type_dir =  path_image.split('/')[-2]

    # image_save_dir = path_image.replace('hole/images',data_type+'_hole/images').replace(data_type_dir+'/'+image_name,'')  ##

    # raw_image

    image = cv2.imread(path_image)
    image_mask = cv2.imread(path_mask,0)
    image_save_dir = '/home/gzr/Data/hole_data/hole_dataset/'+data_type+'_hole/images/test/raw'

    # crop_augument

    image_save_dir = '/home/gzr/Data/hole_data/hole_dataset/'+data_type+'_hole/images/test/crop'


    #










for data_type in range(7,8):
    num_test = 50
    cnt=0
    path_mask_list = []
    file_image_list = []

    dir_images = image_base + 'data_' + str(data_type)
    files = os.listdir(dir_images)
    for file in files:
        if 'mask' not in file:
            continue
        path_mask =  os.path.join(dir_images,file)
        file_image = file.replace('_mask','')
        path_image = os.path.join(dir_images,file_image)
        path_mask_list.append(path_mask)
        file_image_list.append(path_image)

    dir_images = image_base_1 + 'data_' + str(data_type)
    files = os.listdir(dir_images)
    for file in files:
        if 'mask' not in file:
            continue
        path_mask =  os.path.join(dir_images,file)
        file_image = file.replace('_mask','')
        path_image = os.path.join(dir_images,file_image)
        path_mask_list.append(path_mask)
        file_image_list.append(path_image)

    dir_images = image_base_2 + 'data_' + str(data_type)
    files = os.listdir(dir_images)
    for file in files:
        if 'mask' not in file:
            continue
        path_mask =  os.path.join(dir_images,file)
        file_image = file.replace('_mask','')
        path_image = os.path.join(dir_images,file_image)
        path_mask_list.append(path_mask)
        file_image_list.append(path_image)


    is_ellipse = data_type<7



    print(len(file_image_list))

    list = np.random.choice(contours_points_fine.shape[0], num_pad, replace=True)

    for i in range(len(file_image_list)):
        deal_sample(path_mask,path_image,is_ellipse,str(data_type),cnt)
        cnt+=1













1682
230
158
8802


KeyboardInterrupt: 

In [28]:

#

TypeError: can only concatenate str (not "list") to str